## 插件

Nornir 是一个插件式系统，nornir3 中只包含了非常基础的插件，可以访问 [nornir.tech](https://nornir.tech/nornir/plugins) 来查看已经公开发布的第三方插件，本篇介绍 nornir 自带的基础插件。

### 注册插件

从 nornir3 开始，插件需要注册之后才能使用，包括：

   - 主机清单插件（inventory plugins）
   - 转换函数（transform functions）
   - 连接插件（connection plugins）
   - 运行插件（runners）

插件注册有两种方式：

1. 在打包发布插件时添加 [entry point](https://setuptools.readthedocs.io/en/latest/setuptools.html#dynamic-discovery-of-services-and-plugins) 来让 nornir 自动注册
2. 在脚本中编写代码手动注册


#### 自动注册

如果使用 `setuptools` 打包发布，需要在 `setup.py` 中添加 `entry_points`：

```python
setup(
    # ...
    entry_points={
      "PATH": "NAME = path.to:Plugin",
    }
)
```

如果使用 `peotry` 打包发布，需要在 pyproject.toml 中添加配置：

```python
[tool.poetry.plugins."PATH"]
"NAME" = "path.to:Plugin"
```


其中：

`PATH` 是需要配置的插件类型，有以下几个值：

   - `nornir.plugins.inventory` - for inventory plugins
   - `nornir.plugins.transform_function` - for transform functions
   - `nornir.plugins.runners` - for runners
   - `nornir.plugins.connections` - for connection plugins

`NAME` 是要引用插件的的名称， `path.to:Plugin` 是导入路径，例如定义一个名为 `inventory-name` 的主机清单插件：

```python
[tool.poetry.plugins."nornir.plugins.inventory"]
"inventory-name" = "path.to:InventoryPlugin"
```

使用带有 `entry point` 打包发布的插件，使用时 nornir 会自动进行注册。

#### 手动注册

如果需要使用本地编写的插件，或者发布时没有添加 `entry point` 的插件，需要以代码的方式来注册，例如注册一个名为 `inventory-name` 的主机清单插件：

```python
from nornir.core.plugins.inventory import InventoryPluginRegister
from path.to import InventoryPlugin

InventoryPluginRegister.register("inventory-name", InventoryPlugin)
```


### 连接插件（Connections）

是一种让 nornir 管理与设备的连接的插件，常见的有 netmiko、paramiko、napalm 等。


### 主机清单插件（Inventory）

是一种让 nornir 从外部源（如 YAML、CSV、DB 等）创建一个 `Inventory` 对象的插件，常见的有 SimpleInventory、ansible、netbox、table 等。

#### SimpleInventory

Nornir 默认包含 SimpleInventory 插件，它从 YAML 文件读取信息，源码定义如下：

```python
class nornir.plugins.inventory.__init__.SimpleInventory(
    host_file: str = 'hosts.yaml', 
    group_file: str = 'groups.yaml', 
    defaults_file: str = 'defaults.yaml', 
    encoding: str = 'utf-8'
):

    def load()-> nornir.core.inventory.Inventory:...
```


### 转换函数（Transform function）

是一种独立于已使用的主机清单插件来操作主机的插件，可以帮助用户使用环境变量、加密存储或者类似的方式来对主机的数据进行扩展。


### 运行插件（Runners）

是一种定义如何在主机上执行任务的插件，分为单线程和多线程。

#### SerialRunner

SerialRunner 单线程插件，在每个主机上顺序执行任务，源码定义如下：

```python
class nornir.plugins.runners.__init__.SerialRunner():
    
    def run(
        task: nornir.core.task.Task, 
        hosts: List[nornir.core.inventory.Host]
    ) -> nornir.core.task.AggregatedResult: ...

```

#### ThreadedRunner

ThreadedRunner 使用多线程在每个主机上异步执行任务，源码定义如下：

```python
class nornir.plugins.runners.__init__.ThreadedRunner(num_workers: int = 20):
    """Parameters:
    num_workers – number of threads to use"""
    
    def run(
        task: nornir.core.task.Task,
        hosts: List[nornir.core.inventory.Host]
    ) -> nornir.core.task.AggregatedResult: ...
```

有关 ThreadedRunner 的更多详细信息，请阅读 [任务执行模型](./execution_model.rst)。

### 处理器（Processors）

是一种让用户编写任意代码来控制任务执行事件的插件，具体使用方法见入门教程中的 [处理器](../tutorial/09.processors.ipynb)。
